# Fallacy detection

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/mnt/c/users/evacantin/desktop/MASTER/HAIA/Trabajo/afd_text_audio_train.csv')
df['Etiqueta'] = df['Etiqueta'].astype(int)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Etiqueta'], random_state=42)

In [2]:
import torch
from torch.utils.data import Dataset
import torchaudio
from transformers import RobertaTokenizer, Wav2Vec2Processor

text_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
audio_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

MAX_AUDIO_LEN = 160000  # recorte opcional (~10 seg a 16kHz)

class MultiModalDataset(Dataset):
    def __init__(self, df, text_tokenizer, audio_processor):
        self.df = df.reset_index(drop=True)
        self.text_tokenizer = text_tokenizer
        self.audio_processor = audio_processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        texto = row['Texto']
        ruta_audio = row['Ruta']
        etiqueta = row['Etiqueta']

        # Tokenizar texto
        text_enc = self.text_tokenizer(
            texto,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        # Cargar audio
        waveform, sample_rate = torchaudio.load(ruta_audio)
        waveform = waveform[:, :MAX_AUDIO_LEN]  # recorte opcional

        audio_enc = audio_processor(
            waveform.squeeze().numpy(),
            sampling_rate=sample_rate,
            return_tensors="pt"
        )

        return {
            "input_ids": text_enc["input_ids"].squeeze(0),
            "attention_mask": text_enc["attention_mask"].squeeze(0),
            "audio_input_values": audio_enc["input_values"].squeeze(0),
            "label": torch.tensor(etiqueta, dtype=torch.long)
        }

In [3]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    audio_input_values = [item['audio_input_values'] for item in batch]
    labels = [item['label'] for item in batch]

    input_ids = torch.stack(input_ids)
    attention_mask = torch.stack(attention_mask)
    labels = torch.stack(labels)

    audio_input_values = pad_sequence(audio_input_values, batch_first=True)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "audio_input_values": audio_input_values,
        "label": labels
    }

In [4]:
import torch.nn as nn
from transformers import RobertaModel, Wav2Vec2Model

class MultiModalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.text_encoder = RobertaModel.from_pretrained("roberta-base")
        self.audio_encoder = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

        self.classifier = nn.Sequential(
            nn.Linear(self.text_encoder.config.hidden_size + self.audio_encoder.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2)
        )
        
        # 🔒 Congelamos los encoders al inicio
        for param in self.text_encoder.parameters():
            param.requires_grad = False

        for param in self.audio_encoder.parameters():
            param.requires_grad = False

    def forward(self, input_ids, attention_mask, audio_input_values):
        text_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        audio_out = self.audio_encoder(audio_input_values).last_hidden_state.mean(dim=1)

        combined = torch.cat((text_out, audio_out), dim=1)
        logits = self.classifier(combined)
        return logits

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datasets and Loaders
train_dataset = MultiModalDataset(train_df, text_tokenizer, audio_processor)
val_dataset = MultiModalDataset(val_df, text_tokenizer, audio_processor)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)

# Model, Loss, Optimizer
model = MultiModalClassifier().to(device)

# Stronger class weights
class_weights = torch.tensor([1.0, 50.0])  # Adjust this if necessary
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Learning rate scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# Training loop
EPOCHS = 10
best_model_state = None
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    print(f"\n======== Epoch {epoch+1}/{EPOCHS} ========")
    
    model.train()
    total_loss = 0
    
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        audio_input_values = batch['audio_input_values'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, audio_input_values)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            audio_input_values = batch['audio_input_values'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask, audio_input_values)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    print(f"Val Loss: {avg_val_loss:.4f}")

    val_accuracy = accuracy_score(all_labels, all_preds)
    val_f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    val_f1_label_0 = f1_score(all_labels, all_preds, pos_label=0, average='binary')
    val_f1_label_1 = f1_score(all_labels, all_preds, pos_label=1, average='binary')

    print(f"Val Accuracy: {val_accuracy:.4f}")
    print(f"Val F1 (weighted): {val_f1_weighted:.4f}")
    print(f"Val F1 (label 0): {val_f1_label_0:.4f}")
    print(f"Val F1 (label 1): {val_f1_label_1:.4f}")

    # Update learning rate
    scheduler.step(avg_val_loss)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model.state_dict())
        print("🔥 Nuevo mejor modelo guardado.")

    # Print current learning rate
    for param_group in optimizer.param_groups:
        print(f"Current Learning Rate: {param_group['lr']:.6f}")


if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("✅ Best model loaded for evaluation or saving.")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1/10 ========


Training Epoch 1:   0%|          | 6/1712 [00:03<15:44,  1.81it/s]/home/evacantin/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Training Epoch 1: 100%|██████████| 1712/1712 [17:05<00:00,  1.67it/s]


Train Loss: 0.6972
Val Loss: 0.6911
Val Accuracy: 0.6060
Val F1 (weighted): 0.6893
Val F1 (label 0): 0.7415
Val F1 (label 1): 0.1719
🔥 Nuevo mejor modelo guardado.
Current Learning Rate: 0.000020

======== Epoch 2/10 ========


Training Epoch 2:   0%|          | 1/1712 [00:00<17:55,  1.59it/s]/home/evacantin/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Training Epoch 2: 100%|██████████| 1712/1712 [24:01<00:00,  1.19it/s]


...

Train Loss: 0.6913
Val Loss: 0.6888
Val Accuracy: 0.6706
Val F1 (weighted): 0.7373
Val F1 (label 0): 0.7943
Val F1 (label 1): 0.1730
🔥 Nuevo mejor modelo guardado.
Current Learning Rate: 0.000020

======== Epoch 9/10 ========


Training Epoch 9:   2%|▏         | 27/1712 [00:15<14:45,  1.90it/s]/home/evacantin/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Training Epoch 9: 100%|██████████| 1712/1712 [14:10<00:00,  2.01it/s]


Train Loss: 0.6869
Val Loss: 0.6892
Val Accuracy: 0.4866
Val F1 (weighted): 0.5851
Val F1 (label 0): 0.6256
Val F1 (label 1): 0.1831
Current Learning Rate: 0.000020

======== Epoch 10/10 ========


Training Epoch 10:   1%|▏         | 23/1712 [00:09<11:46,  2.39it/s]/home/evacantin/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Training Epoch 10: 100%|██████████| 1712/1712 [14:05<00:00,  2.02it/s]


Train Loss: 0.6910
Val Loss: 0.6936
Val Accuracy: 0.8718
Val F1 (weighted): 0.8571
Val F1 (label 0): 0.9308
Val F1 (label 1): 0.1272
Current Learning Rate: 0.000020
✅ Best model loaded for evaluation or saving.


In [11]:
# Fusionar ambos datasets
full_df = pd.concat([train_df, val_df]).reset_index(drop=True)
full_dataset = MultiModalDataset(full_df, text_tokenizer, audio_processor)
full_loader = DataLoader(full_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Inicializar de nuevo
model = MultiModalClassifier().to(device)
model.load_state_dict(best_model_state)  # arrancar desde el mejor estado encontrado

optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Reentrenar
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in tqdm(full_loader, desc=f"Full Train Epoch {epoch+1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        audio_input_values = batch['audio_input_values'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, audio_input_values)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Full Train Epoch {epoch+1} - Loss: {total_loss / len(full_loader):.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Full Train Epoch 1:   1%|▏         | 28/2140 [00:16<17:08,  2.05it/s]/home/evacantin/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.st

Full Train Epoch 1 - Loss: 0.6883


Full Train Epoch 2: 100%|██████████| 2140/2140 [20:14<00:00,  1.76it/s]


Full Train Epoch 2 - Loss: 0.6891


Full Train Epoch 3: 100%|██████████| 2140/2140 [20:45<00:00,  1.72it/s]


...

Full Train Epoch 8 - Loss: 0.6870


Full Train Epoch 9: 100%|██████████| 2140/2140 [19:54<00:00,  1.79it/s]


Full Train Epoch 9 - Loss: 0.6855


Full Train Epoch 10: 100%|██████████| 2140/2140 [19:21<00:00,  1.84it/s]

Full Train Epoch 10 - Loss: 0.6880


In [12]:
import os
class MultiModalDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, text_tokenizer, audio_processor, is_test=True):
        self.dataframe = dataframe.reset_index(drop=True)
        self.text_tokenizer = text_tokenizer
        self.audio_processor = audio_processor
        self.is_test = is_test

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row['Texto']
        audio_path = row['Ruta']

        text_enc = self.text_tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')

        try:
            if not os.path.exists(audio_path) or os.path.getsize(audio_path) <= 44:
                raise ValueError(f"Invalid audio file: {audio_path}")

            audio_input, _ = torchaudio.load(audio_path)
            audio_enc = self.audio_processor(audio_input.squeeze(0), sampling_rate=16000, return_tensors='pt')
            audio_input_values = audio_enc["input_values"].squeeze(0)
            audio_valid = True

        except Exception as e:
            print(f"Warning: Corrupt or missing audio at index {idx}: {audio_path}. Setting default.")
            audio_input_values = torch.zeros(16000)  # Por ejemplo: 1 segundo de silencio
            audio_valid = False

        return {
            "input_ids": text_enc["input_ids"].squeeze(0),
            "attention_mask": text_enc["attention_mask"].squeeze(0),
            "audio_input_values": audio_input_values,
            "audio_valid": audio_valid  # indicamos si el audio es válido
        }

    def __len__(self):
        return len(self.dataframe)

In [13]:
def collate_fn_test(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    audio_input_values = [item['audio_input_values'] for item in batch]

    input_ids = torch.stack(input_ids)
    attention_mask = torch.stack(attention_mask)

    audio_input_values = pad_sequence(audio_input_values, batch_first=True)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "audio_input_values": audio_input_values,
    }


In [14]:
# Preparar test set
test_df = pd.read_csv('/mnt/c/users/evacantin/desktop/MASTER/HAIA/Trabajo/afd_text_audio_test.csv')
test_dataset = MultiModalDataset(test_df, text_tokenizer, audio_processor, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn_test)

model.eval()
all_test_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting on Test"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        audio_input_values = batch['audio_input_values'].to(device)

        outputs = model(input_ids, attention_mask, audio_input_values)
        preds = torch.argmax(outputs, dim=1)
        all_test_preds.extend(preds.cpu().numpy())

# Agregar predicciones al dataframe original
test_df['Predicción'] = all_test_preds

Predicting on Test:   0%|          | 0/272 [00:00<?, ?it/s]/home/evacantin/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Predicting on Test:  41%|████      | 112/272 [03:46<11:21,  4.26s/it]

Predicting on Test:  42%|████▏     | 113/272 [03:48<09:21,  3.53s/it]

Predicting on Test:  42%|████▏     | 114/272 [03:49<07:09,  2.72s/it]

Predicting on Test:  42%|████▏     | 115/272 [03:50<05:38,  2.16s/it]

Predicting on Test:  43%|████▎     | 116/272 [03:50<04:41,  1.81s/it]

...

Predicting on Test:  99%|█████████▊| 268/272 [09:21<00:03,  1.07it/s]

Predicting on Test:  99%|█████████▉| 269/272 [09:22<00:02,  1.06it/s]

Predicting on Test:  99%|█████████▉| 270/272 [09:23<00:01,  1.05it/s]

Predicting on Test: 100%|█████████▉| 271/272 [09:24<00:00,  1.04it/s]

Predicting on Test: 100%|██████████| 272/272 [09:25<00:00,  2.08s/it]


In [15]:
test_df

,Texto,Ruta,Etiqueta,Predicción
0,We got to take a look at what I was left when ...,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,1
1,We had an economy that was in free fall.,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,0
2,The pandemic was so badly handled.,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,1
3,Many people were dying.,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,1
4,"All he said was, it's not that serious.",/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,0
...,...,...,...,...
2170,She gave a lot of it away to the Taliban.,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,1
2171,She gave it to Afghanistan.,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,1
2172,What these people have done to our country and...,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,1
2173,Many of them are criminals and they're destroy...,/mnt/c/users/evacantin/desktop/MASTER/HAIA/Tra...,NaN,1


In [16]:
test_df.Predicción.value_counts()

Predicción
1    1190
0     985
Name: count, dtype: int64

In [17]:
test_df.to_csv("afd_roberta_text_audio.csv", index=False)